## 이미지 데이터 세트 만들기

In [1]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob
import numpy as np

In [10]:
## 분류 데이터 로딩
root_dir = "train/"

In [4]:
# 카테고리 명령 지정 (1)
categories = ["아이스아메리카노","시리얼","떡볶이","젤리"]
nb_classes = len(categories)

In [5]:
# 이미지 크기 지정 (2)
image_width = 64
image_height = 64

In [11]:
## 데이터 변수

X = [] # 이미지 데이터
Y = [] # 레이블 데이터

for idx, category in enumerate(categories):
    image_dir = root_dir + category
    files = glob.glob(image_dir + "/" + "*.jpg")
    print(image_dir + "/" +"*.jpg")

    for i, f in enumerate(files):
        # 이미지 로딩 (3)
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_width, image_height))
        data = np.asarray(img)
        X.append(data)
        Y.append(idx)

X = np.array(X)
Y = np.array(Y)

train/아이스아메리카노/*.jpg
train/시리얼/*.jpg
train/떡볶이/*.jpg
train/젤리/*.jpg


In [12]:
X

array([[[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        ...,

        [[217, 217, 217],
         [217, 217, 217],
         [216, 216, 216],
         ...,
         [215, 215, 215],
         [215, 215, 215],
         [215, 215, 215]],

        [[215, 215, 215],
         [214, 214, 214],
         [212, 212, 212],
         ...,
         [212, 212, 212],
         [212, 212, 212],
         [212, 212, 212]],

        [[212, 212, 212],
         [209, 209, 209],
         [209, 209, 209],
         ...,
         [209, 209, 209],
        

In [14]:
# 학습 데이터와 테스트 데이터 나누기 (4)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.25)
xy = (X_train, X_test, Y_train, Y_test)

In [15]:
# 데이터 파일 저장 (5)
np.save(root_dir + "food.npy", xy)

## 4종 음식 분류기

In [16]:
## 사용할 모델 라이브러리 import
import sys, os
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


In [54]:
## 초기 설정
root_dir = "./train/"
categories = ["아이스아메리카노","젤리","떡볶이","시리얼"]
nb_classes = len(categories)
image_size = 64

In [55]:
# 데이터 로딩  (1)
def load_dataset():
    x_train, x_test, y_train, y_test = np.load("food.npy", allow_pickle=True)
    x_train = x_train.astype("float") / 256
    x_test = x_test.astype("float") / 256
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    return  x_train, x_test, y_train, y_test

In [64]:
# 모델 구성  (2)
def build_model(in_shape):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='Same', 
                input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy', 
                    optimizer='rmsprop', 
                    metrics=['accuracy'])

    return model

In [65]:
# 모델 학습을 수행하고 저장된 모델을 파일로 저장  (3)
def model_train(x, y):
    model = build_model(x.shape[1:])
    model.fit(x, y, batch_size=32, epochs=30)

    return model

In [66]:
# 모델 평가하기  (4)
def model_eval(model, x, y):
    score = model.evaluate(x, y)
    print('loss=', score[0])
    print('accuracy=', score[1])

In [67]:
# 모델 학습 및 평가
x_train, x_test, y_train, y_test = load_dataset()
model = model_train(x_train, y_train)
model_eval(model, x_test, y_test)

C:\Users\TJ\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="Same")`
  """
C:\Users\TJ\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  if __name__ == '__main__':
C:\Users\TJ\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/30
126/126 [==============================] - 1s 9ms/step - loss: 2.1093 - accuracy: 0.6687
Epoch 2/30
126/126 [==============================] - 1s 8ms/step - loss: 1.4093 - accuracy: 0.6310
Epoch 3/30
126/126 [==============================] - 1s 8ms/step - loss: 0.5699 - accuracy: 0.7500
Epoch 4/30
126/126 [==============================] - 1s 8ms/step - loss: 0.5601 - accuracy: 0.7500
Epoch 5/30
126/126 [==============================] - 1s 9ms/step - loss: 0.5566 - accuracy: 0.7500
Epoch 6/30
126/126 [==============================] - 1s 8ms/step - loss: 0.5524 - accuracy: 0.7500
Epoch 7/30
126/126 [==============================] - 1s 8ms/step - loss: 0.5433 - accuracy: 0.7500
Epoch 8/30
126/126 [==============================] - 1s 8ms/step - loss: 0.5601 - accuracy: 0.7460
Epoch 9/30
126/126 [==============================] - 1s 8ms/step - loss: 0.5378 - accuracy: 0.7500
Epoch 10/30
126/126 [==============================] - 1s 8ms/step - loss: 0.5412 - accuracy: 0.7460

In [70]:
# 모델 저장
model.save("images.h5")

## 만들어진 모델을 활용한 음식4종 분류하기

In [71]:
import sys, os
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.models import load_model
from PIL import Image
import numpy as np

In [74]:
# 테스트 이미지 목록 (1)
image_files = ["test/떡볶이/떡볶이10.jpg",
               "test/떡볶이/떡볶이14.jpg",
               "test/시리얼/시리얼4.jpg",
               "test/젤리/jelly46.jpg",
               "test/아이스아메리카노/아이스아메리카노4.jpg"]
image_size = 64
nb_classes = len(image_files)
categories = ["아이스아메리카노","젤리","떡볶이","시리얼"]

In [75]:
X = []
files = []
# 이미지 불러오기 (2)
for fname in image_files:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((image_size, image_size))
    in_data = np.asarray(img)
    in_data = in_data.astype("float") / 256
    X.append(in_data)
    files.append(fname)

X = np.array(X)

In [76]:
# 모델 파일 읽어오기  (3)
model = load_model('images.h5')

In [77]:
# 예측 실행  (4)
pre = model.predict(X)

In [78]:
# 예측 결과 출력 (5)
for i, p in enumerate(pre):
    y = p.argmax()
    print("입력:", files[i])
    print("예측:", "[", y, "]", categories[y], "/ Score",p[y])

입력: test/떡볶이/떡볶이10.jpg
예측: [ 2 ] 떡볶이 / Score 0.99963725
입력: test/떡볶이/떡볶이14.jpg
예측: [ 3 ] 시리얼 / Score 0.8720589
입력: test/시리얼/시리얼4.jpg
예측: [ 1 ] 젤리 / Score 0.9003805
입력: test/젤리/jelly46.jpg
예측: [ 1 ] 젤리 / Score 0.9855378
입력: test/아이스아메리카노/아이스아메리카노4.jpg
예측: [ 3 ] 시리얼 / Score 0.83968383


## 엉망진창 예측이군요!! ##